In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[1]').appName('pyspark_fresh').getOrCreate()

In [3]:
import datetime
data= [{
    "id":1,
    "firstName":"Terry",
    "lastName":"Medhurst",
    "email":"atuny0@sohu.com",
    "phone":Row(phone_number = "+63 791 675 8914",mobile_number = "+63 791 675 8945"),
    "courses" : [1,2],
    "is_customer" : True,
    "amount_paid" : 10200,
    "customer_from" : datetime.date(2022,3,10),
    "last_update_ts" : datetime.datetime(2023,10,20, 15,0,45)
},
{
    "id":2,
    "firstName":"Sheldon",
    "lastName":"Quigley",
    "email":"hbingley1@plala.or.jp",
    "phone":Row(phone_number="+7 813 117",mobile_number="+7 813 119"),
    "courses" : [3],
    "is_customer" : True,
    "amount_paid" : 11000,
    "customer_from" : datetime.date(2022,5,16),
    "last_update_ts" : datetime.datetime(2023,10,20, 15,0,45)
},
{
    "id":3,
    "firstName":"Terrill",
    "lastName":"Hills",
    "email":"rshawe2@51.la",
    "phone":Row(phone_number="+63 739 292 7942",mobile_number="+63 739 292 7943"),
    "courses" : [2,4],
    "is_customer" : True,
    "amount_paid" : 24000,
    "customer_from" : datetime.date(2022,4,11),
    "last_update_ts" : datetime.datetime(2023,10,20, 15,0,45)
},
{
    "id":4,
    "firstName":"Miles",
    "lastName":"Cummerata",
    "email":"yraigatt3@nature.com",
    "phone":Row(phone_number=None,mobile_number=None),
    "courses" : [],
    "is_customer" : False,
    "amount_paid" : None,
    "customer_from" : None,
    "last_update_ts" : datetime.datetime(2023,10,20, 15,0,45)
},
{
    "id":5,
    "firstName":"Mavis",
    "lastName":"Schultz",
    "email":"kmeus4@upenn.edu",
    "phone":Row(phone_number="+372 285 771 1911",mobile_number=None),
    "courses" : [],
    "is_customer" : False,
    "amount_paid" : None,
    "customer_from" : None,
    "last_update_ts" : datetime.datetime(2023,10,20, 15,0,45)
}]

In [4]:
import pandas as pd

In [5]:
df = spark.createDataFrame(pd.DataFrame(data))

In [8]:
from pyspark.sql.functions import col

In [6]:
df['id']

Column<'id'>

In [10]:
col('id')

Column<'id'>

In [11]:
type(df['id'])

pyspark.sql.column.Column

In [12]:
type(col('id'))

pyspark.sql.column.Column

In [13]:
df.select(df['id'],col('firstName'),'lastName').show()

+---+---------+---------+
| id|firstName| lastName|
+---+---------+---------+
|  1|    Terry| Medhurst|
|  2|  Sheldon|  Quigley|
|  3|  Terrill|    Hills|
|  4|    Miles|Cummerata|
|  5|    Mavis|  Schultz|
+---+---------+---------+



In [15]:
df.alias('u').select('u.id',col('firstName'),'lastName').show()

+---+---------+---------+
| id|firstName| lastName|
+---+---------+---------+
|  1|    Terry| Medhurst|
|  2|  Sheldon|  Quigley|
|  3|  Terrill|    Hills|
|  4|    Miles|Cummerata|
|  5|    Mavis|  Schultz|
+---+---------+---------+



In [16]:
from pyspark.sql.functions import col,concat,lit

In [19]:
df.select('id','firstName','lastName',concat(df['firstName'],lit('-'),col('lastName')).alias('fullName')).show()

+---+---------+---------+---------------+
| id|firstName| lastName|       fullName|
+---+---------+---------+---------------+
|  1|    Terry| Medhurst| Terry-Medhurst|
|  2|  Sheldon|  Quigley|Sheldon-Quigley|
|  3|  Terrill|    Hills|  Terrill-Hills|
|  4|    Miles|Cummerata|Miles-Cummerata|
|  5|    Mavis|  Schultz|  Mavis-Schultz|
+---+---------+---------+---------------+



In [20]:
df.alias('u').selectExpr('id','firstName','lastName',"concat(u.firstName,', ',u.lastName) AS fullName").show()

+---+---------+---------+----------------+
| id|firstName| lastName|        fullName|
+---+---------+---------+----------------+
|  1|    Terry| Medhurst| Terry, Medhurst|
|  2|  Sheldon|  Quigley|Sheldon, Quigley|
|  3|  Terrill|    Hills|  Terrill, Hills|
|  4|    Miles|Cummerata|Miles, Cummerata|
|  5|    Mavis|  Schultz|  Mavis, Schultz|
+---+---------+---------+----------------+



In [21]:
df.createOrReplaceTempView('users')

In [25]:
spark.sql("""
SELECT id,firstName,lastName,
concat(u.firstName,', ',u.lastName) AS fullName
From users AS u
""").show()

+---+---------+---------+----------------+
| id|firstName| lastName|        fullName|
+---+---------+---------+----------------+
|  1|    Terry| Medhurst| Terry, Medhurst|
|  2|  Sheldon|  Quigley|Sheldon, Quigley|
|  3|  Terrill|    Hills|  Terrill, Hills|
|  4|    Miles|Cummerata|Miles, Cummerata|
|  5|    Mavis|  Schultz|  Mavis, Schultz|
+---+---------+---------+----------------+



### col function Other uses -

## There are quite a few function available on top of column type
    cost(can be used on all important dataframe functions such as select, filter, groupby, orderby etc)
    asc,dsc(typically used as part of sort or orderBy)
    contains( typically used as filter or where)

In [26]:
df.select('id','customer_from').show()

+---+-------------+
| id|customer_from|
+---+-------------+
|  1|   2022-03-10|
|  2|   2022-05-16|
|  3|   2022-04-11|
|  4|         null|
|  5|         null|
+---+-------------+



In [28]:
from pyspark.sql.functions import date_format

In [29]:
df.select(col('id'),date_format('customer_from','yyyyMMdd')).show()

+---+------------------------------------+
| id|date_format(customer_from, yyyyMMdd)|
+---+------------------------------------+
|  1|                            20220310|
|  2|                            20220516|
|  3|                            20220411|
|  4|                                null|
|  5|                                null|
+---+------------------------------------+



In [30]:
df.select(col('id'),date_format('customer_from','yyyyMMdd').alias('customer_from')).show()

+---+-------------+
| id|customer_from|
+---+-------------+
|  1|     20220310|
|  2|     20220516|
|  3|     20220411|
|  4|         null|
|  5|         null|
+---+-------------+



In [33]:
df.select(col('id'),date_format('customer_from','yyyyMMdd').cast('int').alias('customer_from')).show()

+---+-------------+
| id|customer_from|
+---+-------------+
|  1|     20220310|
|  2|     20220516|
|  3|     20220411|
|  4|         null|
|  5|         null|
+---+-------------+



In [35]:
cols = [col('id'),date_format('customer_from','yyyyMMdd').cast('int').alias('customer_from')]

df.select(*cols).show()

+---+-------------+
| id|customer_from|
+---+-------------+
|  1|     20220310|
|  2|     20220516|
|  3|     20220411|
|  4|         null|
|  5|         null|
+---+-------------+



In [36]:
df.select(cols).show()

+---+-------------+
| id|customer_from|
+---+-------------+
|  1|     20220310|
|  2|     20220516|
|  3|     20220411|
|  4|         null|
|  5|         null|
+---+-------------+

